In [1]:
import os
import json
import torch
import numpy as np
from pycocotools.coco import COCO
from PIL import Image

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
data_dir = '/opt/ml/input/mmseg'
annotation_dir = os.path.join(data_dir, 'annotations')
for path in ['train', 'valid']:
    os.makedirs(os.path.join(annotation_dir, path), exist_ok=True)


In [3]:

with open('/opt/ml/input/data/cv_train_1.json','r',encoding='utf8') as f:
    train_data = json.load(f)
train_anns_list=[]
anns=train_data['annotations'][:]
for img_info in train_data['images']:
    tmp_anns=[]
    while anns:
        if anns[0]['image_id']!=img_info['id']:
            tmp_anns=sorted(tmp_anns,key=lambda x:x['area'],reverse=True)
            if tmp_anns:
                train_anns_list.append(tmp_anns)
            break
        tmp_anns.append(anns.pop(0))

with open('/opt/ml/input/data/cv_val_1.json','r',encoding='utf8') as f:
    val_data = json.load(f)
val_anns_list=[]
anns=val_data['annotations'][:]
for img_info in val_data['images']:
    tmp_anns=[]
    while anns:
        if anns[0]['image_id']!=img_info['id']:
            tmp_anns=sorted(tmp_anns,key=lambda x:x['area'],reverse=True)
            if tmp_anns:
                val_anns_list.append(tmp_anns)
            break
        tmp_anns.append(anns.pop(0))


In [4]:
coco_train = COCO('/opt/ml/input/data/cv_train_1.json')
coco_val = COCO('/opt/ml/input/data/cv_val_1.json')
print(len(train_anns_list),len(val_anns_list))


loading annotations into memory...
Done (t=3.15s)
creating index...
index created!
loading annotations into memory...
Done (t=0.65s)
creating index...
index created!
2616 653


In [5]:
import os.path as osp
import numpy as np
from PIL import Image
import mmcv
# convert dataset annotation to semantic segmentation map
data_root = 'iccv09Data'
img_dir = 'images'
ann_dir = 'labels'
# define class and plaette for better visualization
classes = ('Background','General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing')
palette = [[0,0,0],[192,0,128],[0,128,192],[0,128,64],[128,0,0],[64,0,128],[64,0,192],[192,128,64],[192,192,128],[64,64,128],[128,0,192]]


In [10]:
for idx, (anns) in enumerate(train_anns_list):
    masks=np.zeros((512,512))
    for i in range(len(anns)):
        for j,test in enumerate(coco_train.annToMask(anns[i])):
            masks[coco_train.annToMask(anns[i]) == 1] = anns[i]['category_id']
        masks = masks.astype(np.int8)
    seg_map = masks

    seg_img = Image.fromarray(seg_map).convert('P')
    seg_img.putpalette(np.array(palette, dtype=np.uint8))

    image_infos = anns[0]['image_id']
    # annotation을 image_id로 저장합니다.
    file_dir = f"{annotation_dir}/train/{image_infos:04}.png"
    seg_img.save(file_dir)



[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 6 6 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [107]:
for idx, (anns) in enumerate(val_anns_list):
    masks=np.zeros((512,512))
    for i in range(len(anns)):
        for j,test in enumerate(coco_val.annToMask(anns[i])):
            masks[coco_val.annToMask(anns[i]) == 1] = anns[i]['category_id']
        masks = masks.astype(np.int8)
    seg_map = masks
    seg_img = Image.fromarray(seg_map).convert('P')
    seg_img.putpalette(np.array(palette, dtype=np.uint8))

    image_infos = anns[0]['image_id']
    # annotation을 image_id로 저장합니다.
    file_dir = f"{annotation_dir}/valid/{image_infos:04}.png"
    seg_img.save(file_dir)